# Pylos : an extension Object System for Python

> (C) 2016 F. Peschanski under the MIT License

`Pylos` is a simple (one file!) yet useful extension of the Python object system to integrate **generic methods** as in CLOS (the Common Lisp Object System).

This document is a *tutorial* motivating and showing the basic usage of `Pylos`.


In [1]:
import sys
sys.path.append("../src")  # point to where the pylos.py file is

## The (untyped) expression problem illustrated

If like me you have an experience in object-oriented programming (classical, imperative obviously since we are talking in and about Python) as well as in functional programming (in a language like Lisp, Ocaml, Haskell), you probably noticed a tension between :

  - adding new data-structures
  
  - addning new operations working on these data-structures
  
This tension can be illustrated directly in Python, a language supporting both programming styles.

Our case study is a slightly boring example that everyone understands: the *geometrical shapes*.

### The object-oriented programming style

A geometrical shapee in the OO style is an abstract class, e.g. as follows:

In [5]:
class Shape:
    def __init_(self):
        pass
    
    def perimeter(self):
        raise NotImplementedError()

An example of a shape is a rectangle.

In [26]:
class Rectangle(Shape):
    def __init__(self, x, y, w, h):
        self.x = x; self.y = y; self.w = w; self.h = h
        
    def perimeter(self):
        return 2 * (self.w + self.h)
    
    def __repr__(self):
        return "Rectangle(x={}, y={}, w={}, h={})".format(self.x, self.y, self.w, self.h)

Everything's straightforward and we can already "play" with rectangles.

In [27]:
rect = Rectangle(0, 0, 3, 2)
rect

Rectangle(x=0, y=0, w=3, h=2)

In [28]:
rect.perimeter()

10

Adding a new kind of shape is very natural in the OO style.

In [29]:
import math

class Circle(Shape):
    def __init__(self, x, y, r):
        self.x = x; self.y = y; self.r = r
        
    def perimeter(self):        
        return math.pi * 2 * self.r;
    
    def __repr__(self):
        return "Circle(x={}, y={}, r={})".format(self.x, self.y, self.r)

In [30]:
circ = Circle(1, 1, 1)
circ

Circle(x=1, y=1, r=1)

In [31]:
circ.perimeter()

6.283185307179586

The problem arises when one wants to add a new *operation* to the shapes. For example suppose we want to add a translation for shapes.  There's basically only one way to "solve" this problem: add the operation to the base class and *all* it descendants!

In [34]:
def Shape_translate(self, tx, ty):
    raise NotImplementedError()
Shape.translate = Shape_translate

In [35]:
def Point_translate(self, tx, ty):
    self.x += tx
    self.y += ty
Rectangle.translate = Point_translate

In [42]:
Circle.translate = Point_translate

In [43]:
rect

Rectangle(x=2, y=2, w=3, h=2)

In [44]:
rect.translate(2, 2)

In [45]:
rect

Rectangle(x=4, y=4, w=3, h=2)

In [46]:
circ

Circle(x=1, y=1, r=1)

In [47]:
circ.translate(-1, -1)

In [48]:
circ

Circle(x=0, y=0, r=1)

That's a lot work, and it is necessary to "open up" all the existing classes !

### The functional way

In the functional way, it is very easy to add new operations. Let's add a surface computation operation for example, but this time not as a method but as a function. This is clearly not the style by default in Python but it is available (which is a great thing !).

In [52]:
def surface_of_shape(shape):
    if isinstance(shape, Rectangle):
        return shape.w * shape.h
    elif isinstance(shape, Circle):
        return math.pi * shape.r * shape.r
    else:
        raise ValueError("Cannot compute surface of {}".format(shape))

In [53]:
surface_of_shape(rect)

6

In [54]:
surface_of_shape(circ)

3.141592653589793

This time, the real pain is when a new kind of shape must be added, which requires to change *all* the existing operations (those defined as functions).

## A solution in Pylos

There are many solutions to the (untyped) expression problem, which we will not enumerate. But there is at least on language in which the tension between adding structures vs. adding operations never occur: the *Common Lisp Object System* and of course `Pylos` that gets most of its inspiration from it.

So let's first import the (single-file) `Pylos` module.

In [55]:
from pylos import Generic, method

The objectif is to *unify* the concepts of :

  - *instance method* in the OO style
  - and *function* in the functional style
  
For this is introduced the notion of a **generic method**  (a.k.a. generic function). A *generic method* is at the same time :

  - a *method* because it involves a dispatch mechanism
  
  - a *function* because it is defined and is called at the top-level, without a notion of a *receiver*.

In [56]:
surface = Generic("Compute the surface of a shape")

In [57]:
@method(surface)
def surface_of_rectangle(rect : Rectangle):
    return rect.w * rect.h

Signature is: (rect:__main__.Rectangle)
Call arity = 1
Parameter #1: rect
   ==> annot: <class '__main__.Rectangle'>
   ==> kind: POSITIONAL_OR_KEYWORD
      ==> it's a class !


In [61]:
@method(surface)
def surface_of_circle(circ : Circle):
    return math.pi * circ.r * circ.r

Signature is: (circ:__main__.Circle)
Call arity = 1
Parameter #1: circ
   ==> annot: <class '__main__.Circle'>
   ==> kind: POSITIONAL_OR_KEYWORD
      ==> it's a class !


../src/pylos.py:90: GenericWarning: Redefinition of generic call for signature: (circ:__main__.Circle)
  warnings.warn("Redefinition of generic call for signature: {}".format(sig), GenericWarning)


In [62]:
surface(rect)

6

In [63]:
surface(circ)

3.141592653589793

In [64]:
surface(12)

GenericError: Cannot dispatch on argument: 12